# 🚀 How do I send an API request?

1. How can I reverse engineer 'the correct' API request when there is no documentation?

   - Monitor network traffic at 
   - examine the [Java CLI](https://domo-support.domo.com/s/article/360043437733?language=en_US)
   - see [Jae Wilsons TreasureTrove](./README.md#helpful-links)

2. How does the internet handle authentication?
   - pass authentication in cookies
   - pass authentication via request headers
   - different APIs will have different authentication schemes

## ▶️ How to Monitor Network Traffic

1. Go to [Domo > Data > Accounts](https://domo-community.domo.com/datacenter/accounts) to view a list of all accounts you have access to.

2. To Monitor network traffic > Inspect > Network.  

3. Find the API request.
   - Filter for "providers".
   - Examine the URL, headers, request Method (GET, PUT, POST, DELETE) and body<br>
   - Copy the query into a text editor.


### 3 ways to Authenticate the same API request

### ▶️🍪 convert an API request into a function

> Functions improve code legibility, recycle-ability, and maintainability by being *callable* and *parameterized*


1. Implement your functions in `./functions` folder.
2. Use the `requests` library to send an API request to the providers endpoint.
3. modify the below function to receive `headers: dict` and modify the request.
4. test by scraping your browser cookie and passing it into `headers`

`headers = {'cookie': <your_header>}`


In [1]:
# %pip install requests

In [2]:
import requests

# FIX ME -- then move me into ./functions/get_accounts.py
def get_accounts(domo_instance):
    url = f"https://{domo_instance}.domo.com/api/data/v2/datasources/providers"

    return requests.request(method="GET", url=url)

#### Test Implementation of get_accounts

In [3]:
DOMO_INSTANCE = (
    "domo-community"  ## it's common practice to declare constants using all caps.
)
COOKIE = "cookie_goes_here"

In [4]:
# from solutions.get_accounts_v1 import get_accounts

res = get_accounts(domo_instance=DOMO_INSTANCE)
res

<Response [401]>

### 🎓 What is <Response [401]>

1. requests.request [returns an instance of the Response class](https://www.geeksforgeeks.org/response-request-python-requests/) || `requests.models.Response`

2. classes are `dict` that have attributes (fields) and methods (functions)
   - ex. the `Response` class has attributes like `status_code` and method `.json()`

In [5]:
print(type(res))


# from the error message it's clear we need to pass authentication with our API request
print({"status": res.status_code, "is_success": res.ok,})

print("\n")
print({"type of res.json return" : type(res.json())})
res.json()


<class 'requests.models.Response'>
{'status': 401, 'is_success': False}


{'type of res.json return': <class 'dict'>}


{'status': 401,
 'statusReason': 'Unauthorized',
 'path': '/api/data/v2/datasources/providers',
 'message': 'Full authentication is required to access this resource',
 'toe': '9XX4VBORBZ-KTZ59-G3BKH'}

### ▶️ modify `get_accounts` to return a list of account_objects

The `requests.models.Response` class has a method (function), `.json()` that will convert the response into a dictionary.

- How might we test for API errors?
- How might we test for 'logic errors' ex. No accounts returned?

#### sample implementation of ResponseClass

# 🚀 Use Username and Password authentication flow to handle Domo API Authentication

"full authentication" is not the same as client_id and secret authentication which can be retrieved from <https://developer.domo.com>

- client_id and secret auth (same as developer_token authentication) only applies to ["public" APIs](https://www.youtube.com/watch?v=hRwrZABP8RE).

"full authentication" actually has two flavors. (see curl reques examples above)


### ▶️🤐 generate a function, `get_session_token` to handle username and password authentication
- function should receive: `domo_instance: str, domo_username: str, domo_password : str, and return_raw: bool = False`

- `get_session_token` should parse the response and return just the `sessionToken`

🧪 how might we implement tests to ensure expected / assumed behavior
    - can you test for 400 - 500 errors? (res.ok)
    - what if you get a 200 response but sent an invalid password? 

💡 to support debugging, add a test before parsing the response `if return_raw: return res`
    - this way if your API is ever acting strangely you can validate that the API is still returning the same object (i.e. they didn't relable sessionToken to session_token) 



In [6]:
# FIX ME -- then move me into ./functions/auth.py
def get_session_token(domo_instance: str, return_raw: bool = False) -> str:

    url = f"https://{domo_instance}.domo.com/api/content/v2/authentication"

    body = {
        "method": "password",
        "emailAddress": "domo_username",
        "password": "domo_password",
    }

    res = requests.request(method="POST", url=url, json=body, verify=False)

    if return_raw:
        return res

    return res

In [7]:
# from solutions.get_session_token import get_session_token

try:
    get_session_token(
        domo_instance = 'domo-community',
        domo_username = 'dp24@test.com',
        domo_password = 'thisisinsecure',
    )

except Exception as e:
    print(e)

get_session_token() got an unexpected keyword argument 'domo_username'


### ▶️ modify get_accounts() to optionally receive session_token.

1. modify get_accounts to receive `headers: dict = None, session_token : str = None`

2. combine headers and session_token and pass to request

3. retrieve `session_token` from `get_full_auth()` then pass it to your modified `get_accounts`

[solution](./solutions/get_accounts_v2.py)

### ▶️ Put it all together!

In [8]:
# get_accounts goes here



In [9]:
session_token = get_session_token(
    domo_instance = 'domo-community',
    # domo_username = 'dp24@test.com',
    # domo_password = 'thisisinsecure',
)


get_accounts(domo_instance=DOMO_INSTANCE,
            #  session_token=session_token
             )

/home/GitHub/domopalooza-24/.venv/lib/python3.10/site-packages/urllib3/connectionpool.py:1103: InsecureRequestWarning: Unverified HTTPS request is being made to host 'domo-community.domo.com'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


<Response [401]>

# 🧪 Extra Credit - uses classes to introduce consistency to your code

Classes guarantee a consistent response format

Classes allow us to combine data (attributes) and actions (methods)

- create a file `./functions/client.py` 
- create a `@dataclass ResponseClass` with attributes `status: int, is_success: bool , response: Union[dict, str]` 

- add classmethod that converts a `requests.models.Response` into an instance of `ResponseClass`

- modify `get_accounts` to return `ResponseClass`
 
This ensures that each "API function" always returns the same format response (which simplifies testing downstream) even if you switch do a different library (like HTTPX).

[Solution](./solutions/client.py)

In [10]:
# from solutions.client import ResponseClass

from dataclasses import dataclass

# finish me
@dataclass
class ResponseClass:
    status: int
    # parameters go here.

    @classmethod
    def from_request_response(cls, res: requests.models.Response):
        pass



In [11]:
ResponseClass.from_request_response(res)

# 🚀 Solution

In [12]:
from solutions.accounts import get_accounts, Account_Error
from solutions.auth import get_session_token


session_token = get_session_token(
    domo_instance = 'domo-community',
    domo_username = 'dp24@test.com',
    domo_password = 'thisisinsecure',
)

try:
    get_accounts(domo_instance=DOMO_INSTANCE,
                session_token=session_token,
                debug_api= False
                )
except Account_Error as e:
    print(e)

get_accounts | no accounts returned, does this user have access to accounts Domo > Data > Accounts 
